In [1]:
from collections import namedtuple
from dataclasses import dataclass
from typing import List

In [2]:
def hex2bin(s):
    out = ""
    for c in s.lower():
        i = int(c, 16)
        #print(i)
        out += format(i, '0>4b')
    return out

hex2bin("D2FE28")

'110100101111111000101000'

In [11]:
@dataclass
class Packet:
    version: int
    typeid: int
    length: int
        
@dataclass
class Literal(Packet):
    number: int
        
    def value(self):
        return self.number
        
@dataclass
class Operator(Packet):
    subpackets: List[Packet]
    
    def value(self):
        if self.typeid == 0: 
            # sum
            return sum((p.value() for p in self.subpackets))
        elif self.typeid == 1: 
            # product
            out = 1
            for p in self.subpackets:
                out *= p.value()
            return out
        elif self.typeid == 2:
            # min
            return min((p.value() for p in self.subpackets))
        elif self.typeid == 3:
            # max
            return max((p.value() for p in self.subpackets))
        elif self.typeid == 5:
            # greater than
            a, b = self.subpackets[0:2]
            return int(a.value() > b.value())
        elif self.typeid == 6:
            # less than
            a, b = self.subpackets[0:2]
            return int(a.value() < b.value())
        elif self.typeid == 7:
            # equals
            a, b = self.subpackets[0:2]
            return int(a.value() == b.value())
        
def take(s, n, i):
    return s[i:i+n], i+n

def takeint(s, n, i):
    t, i = take(s, n, i)
    return int(t, 2), i

class Stream:
    def __init__(self, hexstr):
        self.hexstr = hexstr
        self.binstr = hex2bin(hexstr)
        self.tree, self.extra = self.__parse_stream(0)
        
    def __parse_stream(self, i0=0):
        s = self.binstr
        i = i0
        v, i = takeint(s, 3, i)
        t, i = takeint(s, 3, i)
        if t == 4:  # Literal
            number = ""
            while True:
                group, i = take(s, 5, i)
                #print(group)
                number += group[1:]
                if group[0] == '0':
                    break
            return Literal(v, t, i-i0, int(number, 2)), i
        else: # Operator
            lengthid, i = takeint(s, 1, i)
            subpackets = []
            if lengthid == 0:
                l, i = takeint(s, 15, i)
                while l > 0:
                    sub, i = self.__parse_stream(i)
                    subpackets.append(sub)
                    l -= sub.length
            else:
                n, i = takeint(s, 11, i)
                subpackets = []
                for _ in range(n):
                    sub, i = self.__parse_stream(i)
                    subpackets.append(sub)
            return Operator(v, t, i-i0, subpackets), i              
    
    def show_tree(self):
        self.show_node(self.tree, "")
    
    def show_node(self, node, indent=""):
        if isinstance(node, Literal):
            print(indent+"Literal(version={0.version} typeid={0.typeid} number={0.number})".format(node))
        elif isinstance(node, Operator):
            print(indent+"Operator(version={0.version} typeid={0.typeid} npackets={1})".format(node, len(node.subpackets)))
            for s in node.subpackets:
                self.show_node(s, indent+"  ")
        else:
            pass
    
    def sum_version(self, node=None):
        if node is None:
            node = self.tree
        v = node.version
        if isinstance(node, Operator):
            for sub in node.subpackets:
                v += self.sum_version(sub)
        return v

def test(name, hexstr):
    print()
    print("-- {} --".format(name))
    s = Stream(hexstr)
    print(s.hexstr)
    print(s.binstr)
    s.show_tree()
    print("sumver = {}".format(s.sum_version()))
    print("value  = {}".format(s.tree.value()))

test("t1", "D2FE28")
test("t2", "38006F45291200")
test("t3", "EE00D40C823060")
test("t4", "8A004A801A8002F478")
test("t5", "620080001611562C8802118E34")
test("t6", "C0015000016115A2E0802F182340")
test("t7", "A0016C880162017C3686B18A3D4780")


-- t1 --
D2FE28
110100101111111000101000
Literal(version=6 typeid=4 number=2021)
sumver = 6
value  = 2021

-- t2 --
38006F45291200
00111000000000000110111101000101001010010001001000000000
Operator(version=1 typeid=6 npackets=2)
  Literal(version=6 typeid=4 number=10)
  Literal(version=2 typeid=4 number=20)
sumver = 9
value  = 1

-- t3 --
EE00D40C823060
11101110000000001101010000001100100000100011000001100000
Operator(version=7 typeid=3 npackets=3)
  Literal(version=2 typeid=4 number=1)
  Literal(version=4 typeid=4 number=2)
  Literal(version=1 typeid=4 number=3)
sumver = 14
value  = 3

-- t4 --
8A004A801A8002F478
100010100000000001001010100000000001101010000000000000101111010001111000
Operator(version=4 typeid=2 npackets=1)
  Operator(version=1 typeid=2 npackets=1)
    Operator(version=5 typeid=2 npackets=1)
      Literal(version=6 typeid=4 number=15)
sumver = 16
value  = 15

-- t5 --
620080001611562C8802118E34
01100010000000001000000000000000000101100001000101010110001011001000100000

In [12]:
inptext = ""
with open("input.txt", "r") as fh:
    for l in fh:
        inptext += l.strip()
inp = Stream(inptext)
inp.show_tree()
inp.sum_version()

Operator(version=4 typeid=0 npackets=53)
  Operator(version=2 typeid=1 npackets=2)
    Operator(version=4 typeid=6 npackets=2)
      Literal(version=0 typeid=4 number=79)
      Literal(version=4 typeid=4 number=18)
    Literal(version=5 typeid=4 number=130)
  Operator(version=6 typeid=1 npackets=2)
    Operator(version=6 typeid=6 npackets=2)
      Literal(version=2 typeid=4 number=17)
      Literal(version=6 typeid=4 number=3739)
    Literal(version=1 typeid=4 number=26181)
  Operator(version=7 typeid=3 npackets=2)
    Literal(version=7 typeid=4 number=3381457610)
    Literal(version=3 typeid=4 number=13)
  Operator(version=4 typeid=0 npackets=4)
    Literal(version=2 typeid=4 number=368803)
    Literal(version=1 typeid=4 number=46)
    Literal(version=7 typeid=4 number=30827667693)
    Literal(version=3 typeid=4 number=172659535)
  Operator(version=3 typeid=1 npackets=2)
    Operator(version=6 typeid=6 npackets=2)
      Literal(version=3 typeid=4 number=30901)
      Literal(version=6 

971

In [13]:
inp.tree.value()

831996589851